In [1]:
!pip install pymorphy2
!pip install -U pymorphy2-dicts-ru

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
from nltk import sent_tokenize
import pymorphy2
from pymorphy2 import MorphAnalyzer
morph = pymorphy2.MorphAnalyzer()
from tqdm.auto import tqdm
tqdm.pandas()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dvb07\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [1]:
medical_dictionary = {'преморбидный фон': 'преморбидный_фон', 
                      'полиорганная недостаточность': 'полиорганная_недостаточность', 
                      'мультисистемный воспалительный синдром': 'мультисистемный_воспалительный_синдром', 
                      'новая коронавирусная инфекция': 'новый_коронавирусный_инфекция', 
                      'медицинские высшие учебные заведения': 'медицинский_высокий_учебный_заведение', 
                      'длительный COVID-синдром': 'длительный_COVID-синдром', 
                      'пост-COVID-синдром': 'пост-COVID-синдром', 
                      'факторы риска': 'фактор_риск', 
                      'генетическая предрасположенность': 'генетический_предрасположенность', 
                      'онкологические заболевания': 'онкологический_заболевание', 
                      'система иммунитета': 'система_иммунитет', 
                      'иммунный ответ': 'иммунный_ответ', 
                      'противовирусная терапия': 'противовирусный_терапия', 
                      'SARS-CoV-2': 'SARS-CoV-2', 
                      'Listeria monocytogenes': 'Listeria_monocytogenes', 
                      'дифференциальная диагностика': 'дифференциальный_диагностика', 
                      'бактериальный гнойный менингит': 'бактериальный_гнойный_менингит', 
                      'листериозный менингоэнцефалит': 'листериозный_менингоэнцефалит', 
                      'иммуносупрессивная терапия': 'иммуносупрессивный_терапия', 
                      'психические нарушения': 'психический_нарушение', 
                      'медицинские работники': 'медицинский_работник', 
                      'моноклональные антитела': 'моноклональный_антитело', 
                      'акушерский риск': 'акушерский_риск', 
                      'медицинская карта': 'медицинская_карта', 
                      'заболевание сердца': 'заболевание_сердца', 
                      'анализ крови': 'анализ_кровь', 
                      'лечение рака': 'лечение_рак', 
                      'хирургическая операция': 'хирургический_операция', 
                      'неврологическое заболевание': 'неврологический_заболевание', 
                      'ортопедическая хирургия': 'ортопедический_хирургия', 
                      'гинекологическое обследование': 'гинекологический_обследование', 
                      'диагностика рака': 'диагностика_рак', 
                      'терапия боли': 'терапия_боль', 
                      'дыхательная система': 'дыхательный_система', 
                      'пищеварительная система': 'пищеварительный_система', 
                      'сердечно-сосудистая система': 'сердечно-сосудистый_система', 
                      'нервная система': 'нервный_система', 
                      'эндокринная система': 'эндокринный_система', 
                      'иммунная система': 'иммунный_система', 
                      'мочевыделительная система': 'мочевыделительный_система', 
                      'операционная палата': 'операционный_палата', 
                      'онкологическое заболевание': 'онкологический_заболевание', 
                      'диабетическая ретинопатия': 'диабетический_ретинопатия', 
                      'анестезиологическое обезболивание': 'анестезиологический_обезболивание', 
                      'коронавирусная инфекция': 'коронавирусный_инфекция', 
                      'респираторный синдром': 'респираторный_синдром', 
                      'пандемия коронавируса': 'пандемия_коронавирус', 
                      'карантинное ограничение': 'карантинный_ограничение', 
                      'социальное дистанцирование': 'социальное_дистанцирование', 
                      'вакцинация от COVID-19': 'вакцинация_от_COVID-19', 
                      'коронавирусный вариант': 'коронавирусный_вариант', 
                      'пневмония из-за коронавируса': 'пневмония_из-за_коронавирус', 
                      'антитела против COVID-19': 'антитело_против_COVID-19', 
                      'терапия кислородом': 'терапия_кислород', 
                      'реанимационная помощь': 'реанимационный_помощь', 
                      'вирус SARS-CoV-2': 'вирус_SARS-CoV-2', 
                      'COVID-19': 'COVID-19', 
                      'карантинное время': 'карантинный_время', 
                      'штамм вируса': 'штамм_вирус', 
                      'профилактические меры': 'профилактический_мера', 
                      'масочный режим': 'масочный_режим', 
                      'симптомы COVID-19': 'симптом_COVID-19', 
                      'дистанцирование': 'дистанцирование', 
                      'респираторные заболевания': 'респираторный_заболевание', 
                      'пневмония': 'пневмония', 
                      'контакт с больным': 'контакт_с_больной', 
                      'передача воздушно-капельным путем': 'передача_воздушно-капельный_путь', 
                      'карантинная изоляция': 'карантинный_изоляция', 
                      'антисептические средства': 'антисептический_средство', 
                      'вакцинация против COVID-19': 'вакцинация_против_COVID-19', 
                      'антигенный тест': 'антигенный_тест', 
                      'реанимационное отделение': 'реанимационный_отделение', 
                      'критическое состояние': 'критический_состояние', 
                      'инкубационный период': 'инкубационный_период', 
                      'респираторный криз': 'респираторный_криз', 
                      'распространение вируса': 'распространение_вирус', 
                      'иммунитет к COVID-19': 'иммунитет_к_COVID-19', 
                      'COVID-19 у детей': 'COVID-19_у_ребенок', 
                      'мутация вируса': 'мутация_вирус', 
                      'антивирусные препараты': 'антивирусный_препарат', 
                      'реабилитация после COVID-19': 'реабилитация_после_COVID-19', 
                      'коронавирусная пандемия': 'коронавирусный_пандемия', 
                      'распространение вируса SARS-CoV-2': 'распространение_вирус_SARS-CoV-2', 
                      'коронавирусные вакцины': 'коронавирусный_вакцина', 
                      'медицинская маска': 'медицинский_маска', 
                      'подтвержденный случай': 'подтвержденный_случай', 
                      'изоляция в домашних условиях': 'изоляция_в_домашний_условие', 
                      'вакцинационный паспорт': 'вакцинационный_паспорт', 
                      'пациент с COVID-19': 'пациент_с_COVID-19', 
                      'переносчик вируса': 'переносчик_вирус', 
                      'линейка по COVID-19': 'линейка_по_COVID-19', 
                      'смертельный исход': 'смертельный_исход', 
                      'дыхательная недостаточность': 'дыхательный_недостаточность', 
                      'медицинский персонал': 'медицинский_персонал', 
                      'COVID-19 у пожилых людей': 'COVID-19_у_пожилой_человек', 
                      'международные ограничения': 'международный_ограничение', 
                      'цифровая медицина': 'цифровой_медицина', 
                      'контроль за заболеваемостью': 'контроль_за_заболеваемость', 
                      'биологический вирусный штамм': 'биологический_вирусный_штамм', 
                      'гипермобильность суставов': 'гипермобильность_сустав', 
                      'болевой суставной синдром': 'болевой_суставной_синдром', 
                      'постковидный период': 'постковидный_период', 
                      'гастроэзофагеальная рефлюксная болезнь': 'гастроэзофагеальный_рефлюксный_болезнь', 
                      'ингибиторы протонной помпы': 'ингибитор_протонный_помпа', 
                      'С-реактивный белок': 'С-реактивный_белок', 
                      'респираторные инфекции': 'респираторный_инфекция', 
                      'клинические рекомендации': 'клинический_рекомендация', 
                      'маркеры воспаления': 'маркер_воспаление', 
                      'когнитивные нарушения': 'когнитивный_нарушение', 
                      'антибактериальная терапия': 'антибактериальный_терапия', 
                      'синдром «сухого глаза»': 'синдром_сухой_глаз', 
                      'поверхность глаза': 'поверхность_глаз', 
                      'липосомальный бычий лактоферрин': 'липосомальный_бычий_лактоферрин', 
                      'комплексное лечение': 'комплексный_лечение', 
                      'ограничение двигательной активности': 'ограничение_двигательный_активность', 
                      'пожилой возраст': 'пожилой_возраст', 
                      'хроническая боль': 'хронический_боль', 
                      'новорожденные дети': 'новорожденный_ребенок', 
                      'кислородные маски': 'кислородный_маска', 
                      'внебольничная пневмония': 'внебольничный_пневмония', 
                      'острый респираторный дистресс-синдром': 'острый_респираторный_дистресс-синдром', 
                      'насыщение крови кислородом': 'насыщение_кровь_кислород', 
                      'ревматические заболевания': 'ревматический_заболевание', 
                      'ревматоидный артрит': 'ревматоидный_артрит', 
                      'анкилозирующий спондилоартрит': 'анкилозирующий_спондилоартрит', 
                      'псориатический артрит': 'псориатический_артрит', 
                      'постковидный синдром': 'постковидный_синдром', 
                      'углубленная диспансеризация': 'углубленный_диспансеризация', 
                      'скелетно-мышечные проявления': 'скелетно-мышечный_проявление', 
                      'кишечные симптомы': 'кишечный_симптом', 
                      'кишечная проницаемость': 'кишечный_проницаемость', 
                      'кишечный микробиом': 'кишечный_микробиом', 
                      'артериит Такаясу': 'артериит_Такаясу', 
                      'неспецифический аортоартериит': 'неспецифический_аортоартериит', 
                      'болезнь Стилла': 'болезнь_Стилл', 
                      'инвазивный микоз': 'инвазивный_микоз', 
                      'прогнозирование эффективности': 'прогнозирование_эффективность', 
                      'противовоспалительная терапия': 'противовоспалительный_терапия', 
                      'продуктивный кашель': 'продуктивный_кашель', 
                      'постинфекционный кашель': 'постинфекционный_кашель', 
                      'мукоактивные препараты': 'мукоактивный_препарат', 
                      'фиксированная комбинация сальбутамола': 'фиксированный_комбинация_сальбутамол', 
                      'диффузионная способность легких': 'диффузионный_способность_легкое', 
                      'компьютерная томография': 'компьютерный_томография', 
                      'предикторы летальности': 'предиктор_летальность', 
                      'предикторы неблагоприятного исхода': 'предиктор_неблагоприятный_исход', 
                      'артериальная гипертензия': 'артериальный_гипертензия', 
                      'генетические факторы': 'генетический_фактор', 
                      'воспалительные заболевания кишечника': 'воспалительный_заболевание_кишечник', 
                      'язвенный колит': 'язвенный_колит', 
                      'болезнь Крона': 'болезнь_Крон', 
                      'менструальная функция': 'менструальный_функция', 
                      'рекомбинантный ДНК': 'рекомбинантный_ДНК', 
                      'полиморфонуклеарные клетки крови': 'полиморфонуклеарный_клетка_кровь', 
                      'мононуклеарные фагоциты крови': 'мононуклеарный_фагоцит_кровь', 
                      'макрофаги в тканях': 'макрофаг_ткань', 
                      'нейтрофильные гранулоциты': 'нейтрофильный_гранулоцит', 
                      'лимфоциты в крови': 'лимфоцит_кровь', 
                      'эозинофильные гранулоциты': 'эозинофильный_гранулоцит', 
                      'базофильные гранулоциты крови': 'базофильный_гранулоцит_кровь', 
                      'дендритные клетки в иммунной системе': 'дендритный_клетка_иммунный_система', 
                      'натуральные убийцы': 'натуральный_убийца', 
                      'тромбоциты крови': 'тромбоцит_кровь', 
                      'гематокритный индекс': 'гематокритный_индекс', 
                      'индекс дыхательной функции': 'индекс_дыхательный_функция', 
                      'вестибулярная система уха': 'вестибулярный_система_ухо', 
                      'артроз костно-суставной системы': 'артроз_костный_суставной_система', 
                      'артериальное давление': 'артериальный_давление', 
                      'функция печени': 'функция_печень', 
                      'эндокринная система организма': 'эндокринный_система_организм', 
                      'микробиом кишечника': 'микробиом_кишечник', 
                      'иммунологическая реактивность': 'иммунологический_реактивность', 
                      'онкологический риск': 'онкологический_риск', 
                      'мутация генома': 'мутация_геном', 
                      'эпидемиологический анализ': 'эпидемиологический_анализ', 
                      'иммуноглобулин в крови': 'иммуноглобулин_кровь', 
                      'биохимический анализ крови': 'биохимический_анализ_кровь', 
                      'цитологическое исследование': 'цитологический_исследование', 
                      'рентгенологическое обследование': 'рентгенологический_обследование', 
                      'гипертонический раствор': 'гипертонический_раствор', 
                      'симптоматическое лечение': 'симптоматический_лечение', 
                      'микроскопический анализ': 'микроскопический_анализ', 
                      'патологическая анатомия': 'патологический_анатомия', 
                      'дерматологическое заболевание': 'дерматологический_заболевание', 
                      'фармакологическое воздействие': 'фармакологический_воздействие', 
                      'иммунологическая реакция': 'иммунологический_реакция', 
                      'неврологические расстройства': 'неврологический_расстройство', 
                      'психосоматическое заболевание': 'психосоматический_заболевание', 
                      'реабилитационная терапия': 'реабилитационный_терапия', 
                      'гинекологическое заболевание': 'гинекологический_заболевание', 
                      'нутрициологический анализ': 'нутрициологический_анализ', 
                      'психофизиологическое состояние': 'психофизиологический_состояние', 
                      'врожденные пороки сердца': 'врожденный_порок_сердце', 
                      'метаболические нарушения': 'метаболический_нарушение', 
                      'патологический процесс': 'патологический_процесс', 
                      'молекулярные механизмы': 'молекулярный_механизм', 
                      'биохимический анализ': 'биохимический_анализ', 
                      'медикаментозное лечение': 'медикаментозный_лечение', 
                      'бактериальная инфекция': 'бактериальный_инфекция', 
                      'вирусные заболевания': 'вирусный_заболевание', 
                      'паразитарные заболевания': 'паразитарный_заболевание', 
                      'физиологические процессы': 'физиологический_процесс', 
                      'психосоматические расстройства': 'психосоматический_расстройство', 
                      'онкогенные мутации': 'онкогенный_мутация', 
                      'эндокринные нарушения': 'эндокринный_нарушение', 
                      'генетические аномалии': 'генетический_аномалия', 
                      'микробные патогены': 'микробный_патоген', 
                      'нутриентные дефициты': 'нутриентный_дефицит', 
                      'психологическое состояние': 'психологический_состояние', 
                      'репродуктивное здоровье': 'репродуктивный_здоровье', 
                      'иммунологический агент': 'иммунологический_агент', 
                      'анатомические структуры': 'анатомический_структура', 
                      'генетические маркеры': 'генетический_маркер', 
                      'эмоциональное благополучие': 'эмоциональный_благополучие', 
                      'демографические данные': 'демографический_данные', 
                      'физическое здоровье': 'физический_здоровье', 
                      'психосоциальная поддержка': 'психосоциальный_поддержка', 
                      'генетическая консультация': 'генетический_консультация', 
                      'медицинский диагноз': 'медицинский_диагноз', 
                      'медицинская интервенция': 'медицинский_интервенция', 
                      'профилактические мероприятия': 'профилактический_мероприятие', 
                      'социальные аспекты': 'социальный_аспект', 
                      'генетический анализ': 'генетический_анализ', 
                      'биомедицинская инженерия': 'биомедицинский_инженерия', 
                      'молекулярная биология': 'молекулярный_биология', 
                      'клеточная биология': 'клеточный_биология', 
                      'геномная медицина': 'геномный_медицина', 
                      'молекулярная диагностика': 'молекулярный_диагностика', 
                      'биомаркеры заболеваний': 'биомаркер_заболевание', 
                      'медицинская генетика': 'медицинский_генетика', 
                      'биохимия клетки': 'биохимия_клетка', 
                      'медицинская биофизика': 'медицинский_биофизика', 
                      'генная терапия': 'генный_терапия', 
                      'геномное секвенирование': 'геномный_секвенирование', 
                      'молекулярные маркеры': 'молекулярный_маркер', 
                      'генетические алгоритмы': 'генетический_алгоритм', 
                      'рекомбинантные препараты': 'рекомбинантный_препарат', 
                      'транскрипционная регуляция': 'транскрипционный_регуляция', 
                      'медицинская микробиология': 'медицинский_микробиология', 
                      'биохимический анализ мочи': 'биохимический_анализ_мочь', 
                      'молекулярный докинг': 'молекулярный_докинг', 
                      'биомедицинская геоника': 'биомедицинский_генетика'
                     }
                      

In [3]:
medical_dictionary

{'преморбидный фон': 'преморбидный_фон',
 'полиорганная недостаточность': 'полиорганная_недостаточность',
 'мультисистемный воспалительный синдром': 'мультисистемный_воспалительный_синдром',
 'новая коронавирусная инфекция': 'новый_коронавирусный_инфекция',
 'медицинские высшие учебные заведения': 'медицинский_высокий_учебный_заведение',
 'длительный COVID-синдром': 'длительный_COVID-синдром',
 'пост-COVID-синдром': 'пост-COVID-синдром',
 'факторы риска': 'фактор_риск',
 'генетическая предрасположенность': 'генетический_предрасположенность',
 'онкологические заболевания': 'онкологический_заболевание',
 'система иммунитета': 'система_иммунитет',
 'иммунный ответ': 'иммунный_ответ',
 'противовирусная терапия': 'противовирусный_терапия',
 'SARS-CoV-2': 'SARS-CoV-2',
 'Listeria monocytogenes': 'Listeria_monocytogenes',
 'дифференциальная диагностика': 'дифференциальный_диагностика',
 'бактериальный гнойный менингит': 'бактериальный_гнойный_менингит',
 'листериозный менингоэнцефалит': 'лист

In [4]:
model_name = "IlyaGusev/rugpt3medium_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

C:\Users\dvb07\anaconda3\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dvb07\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.98M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/105 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/986 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

In [5]:
df = pd.read_csv("C:\\Users\\dvb07\\OneDrive\\VKR\\data_vkr\\clean_corpusdata.csv")

In [33]:
def summarize(article):
        
    def lemmatize(sentence):
        words = sentence.split()
        lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
        return ' '.join(lemmatized_words)
    
    raw_sentences = sent_tokenize(article)
    lemmatized_sentences = [lemmatize(sent) for sent in raw_sentences]
    
    lemmatized_ner_annotated_sentences = []

    for sent in lemmatized_sentences:
        current_sentence = sent
        for ner, replace_ner in medical_dictionary.items():
            current_sentence = current_sentence.replace(ner, replace_ner)
        lemmatized_ner_annotated_sentences.append(current_sentence)
    lemmatized_ner_annotated_sentences = ' '.join([str(tok) for tok in lemmatized_ner_annotated_sentences])
    
    text_tokens = tokenizer(lemmatized_ner_annotated_sentences, max_length=800, add_special_tokens=True, padding=True, truncation=True)["input_ids"]
    input_ids = text_tokens + [tokenizer.sep_token_id]
    input_ids = torch.LongTensor([input_ids])
    
    output_ids = model.generate(input_ids=input_ids, no_repeat_ngram_size=4, max_new_tokens=1000)
    
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
    summary = summary.split(tokenizer.sep_token)[1]
    summary = summary.split(tokenizer.eos_token)[0]
    
    return summary


In [34]:
# model.config

In [35]:
df['summary'] = df['text'].progress_apply(summarize)

  0%|          | 0/123 [00:00<?, ?it/s]

In [36]:
df['summary'].head(5)

0    Психологический благополучие — один из фундаме...
1    Коронавирус — один из самых смертоносных вирус...
2    Иммунитет — это совокупность защитных реакций ...
3    В России в 2022 году ожидается вспышка ОРВИ и ...
4    Лечение и профилактика листериозных инфекций в...
Name: summary, dtype: object

In [37]:
# max_length=600,
df.to_csv('med_gpt.csv')